In [6]:
import numpy as np
import time

import PIL.Image as Image
import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

In [7]:
batch_size = 32
img_height = 224
img_width = 224
data_root = 'Rice/rice_images'

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  str(data_root),
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = np.array(train_ds.class_names)
print('class names for predictions :',class_names)

Found 3355 files belonging to 4 classes.
Using 2684 files for training.
class names for predictions : ['_BrownSpot' '_Healthy' '_Hispa' '_LeafBlast']


In [8]:
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))

In [9]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [10]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(32, 224, 224, 3)
(32,)


In [3]:
import tensorflow_hub as hub
import os
# Set the cache directory for TensorFlow Hub
hub_path = 'path/to/tfhub_cache'
os.environ['TFHUB_CACHE_DIR'] = hub_path

# Now import TensorFlow Hub
import tensorflow_hub as hub


In [4]:
feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
feature_extractor_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=(224, 224, 3), trainable=False)

In [11]:
num_classes = len(class_names)

model = tf.keras.Sequential([
  feature_extractor_layer,
  tf.keras.layers.Dense(num_classes)
])

model.summary()
image_batch.shape

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 4)                 5124      
                                                                 
Total params: 2263108 (8.63 MB)
Trainable params: 5124 (20.02 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________


TensorShape([32, 224, 224, 3])

In [12]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

In [13]:

# Save the model in h5 format
save_path = 'rice.h5'
model.save(save_path)

# Save the model in pbtxt format
model_dir = os.path.dirname(save_path)
model_name = os.path.basename(save_path)

tf.io.write_graph(graph_or_graph_def=tf.compat.v1.get_default_graph(), logdir=model_dir, name=model_name + '.pbtxt', as_text=True)

d:\EDI_FINAL\.venv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


'rice.h5.pbtxt'

In [15]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import tensorflow_hub as hub
from tkinter import ttk
import tkinter as tk

# Define custom objects to be loaded
custom_objects = {'KerasLayer': hub.KerasLayer}

# Load the trained model
model = load_model('models/sugercane_model.h5', custom_objects=custom_objects)


# Define class names
class_names = ['Bacterial Blight', 'Healthy', 'Red Rot'] #Disease

# Preprocess function
def preprocess_image(image_path):
    img = Image.open(image_path)
    img = img.resize((224, 224))  # Resize image to match model input size
    img = np.array(img) / 255.0  # Normalize pixel values
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Prediction function
def predict_disease(image_path):
    img = preprocess_image(image_path)
    prediction = model.predict(img)
    predicted_class_index = np.argmax(prediction)
    predicted_class = class_names[predicted_class_index]
    return predicted_class

# Tkinter UI
class DiseasePredictorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Disease Predictor")
        self.file_path = None  # Initialize file path attribute

        self.image_label = tk.Label(self.root)
        self.image_label.pack()

        self.upload_button = tk.Button(self.root, text="Upload Image", command=self.upload_image)
        self.upload_button.pack()

        self.predict_button = tk.Button(self.root, text="Predict Disease", command=self.predict)
        self.predict_button.pack()

        self.prediction_label = tk.Label(self.root, text="")
        self.prediction_label.pack()

    def upload_image(self):
        self.file_path = filedialog.askopenfilename()
        if self.file_path:
            self.image = Image.open(self.file_path)
            self.image.thumbnail((300, 300))
            self.photo = ImageTk.PhotoImage(self.image)
            self.image_label.config(image=self.photo)

    def predict(self):
        if self.file_path:
            predicted_class = predict_disease(self.file_path)
            self.prediction_label.config(text="Predicted Disease: " + predicted_class)
        else:
            self.prediction_label.config(text="Please upload an image first.")

# Create Tkinter root window
root = tk.Tk()
app = DiseasePredictorApp(root)
root.mainloop()


1/1 [==============================] - 1s 1s/step


In [2]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import tensorflow as tf
import numpy as np
import tensorflow_hub as hub

# Dictionary mapping diseases to recommended pesticides
pesticide_recommendations = {
    'Bacterial Blight': 'Copper-based fungicides, Streptomycin',
    'Red Rot': 'Fungicides containing Mancozeb or Copper',
    'Blight': 'Fungicides containing Chlorothalonil',
    'Common_Rust': 'Fungicides containing Azoxystrobin or Propiconazole',
    'Gray_Leaf_Spot,Healthy': 'Fungicides containing Azoxystrobin or Propiconazole',
    'Bacterial blight': 'Copper-based fungicides, Streptomycin',
    'curl_virus': 'Insecticides such as Imidacloprid or Pyrethroids',
    'fussarium_wilt': 'Soil fumigants, Fungicides containing Thiophanate-methyl',
    'Bacterial_blight': 'Copper-based fungicides, Streptomycin',
    'Blast': 'Fungicides containing Tricyclazole or Propiconazole',
    'Brownspot': 'Fungicides containing Azoxystrobin or Propiconazole',
    'Tungro': 'Insecticides such as Neonicotinoids or Pyrethroids',
    'septoria': 'Fungicides containing Azoxystrobin or Propiconazole',
    'strip_rust': 'Fungicides containing Azoxystrobin or Propiconazole'
}

# Function to recommend pesticide based on predicted disease
def recommend_pesticide(predicted_class):
    if predicted_class == 'Healthy':
        return 'No need for any pesticide, plant is healthy'
    return pesticide_recommendations.get(predicted_class, "No recommendation available")

# Function to load a model with TensorFlow Hub layer
def load_model_with_hub(model_path):
    # Define a custom object scope to include the KerasLayer
    custom_objects = {"KerasLayer": hub.KerasLayer}
    return tf.keras.models.load_model(model_path, custom_objects=custom_objects)

# Load all five models
models = {
    'sugarcane': load_model_with_hub("models/sugercane_model.h5"),
    'maize': load_model_with_hub("models/maize_model.h5"),
    'cotton': load_model_with_hub("models/cotton_model.h5"),
    'rice': load_model_with_hub("models/rice.h5"),
    'wheat': load_model_with_hub("models/wheat_model.h5"),
}

# Class names for each model
class_names = {
    'sugarcane': ['Bacterial Blight', 'Healthy', 'Red Rot'],
    'maize': ['Blight', 'Common_Rust', 'Gray_Leaf_Spot,Healthy'],  # Replace with your actual class names
    'cotton': ['Bacterial blight','curl_virus', 'fussarium_wilt', 'Healthy'],  # Replace with your actual class names
    'rice': ['Bacterial_blight', 'Blast', 'Brownspot', 'Tungro'],  # Replace with your actual class names
    'wheat': ['Healthy', 'septoria', 'strip_rust'],  # Replace with your actual class names
}

# Function to preprocess an input image
def preprocess_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create batch axis
    img_array /= 255.0  # Normalize the image
    return img_array

# Function to classify an image using the selected model
def classify_image(model_name, file_path):
    input_image = preprocess_image(file_path)
    predictions = models[model_name].predict(input_image)
    predicted_class = class_names[model_name][np.argmax(predictions)]
    recommended_pesticide = recommend_pesticide(predicted_class)
    result_label.config(text=f"Predicted Class ({model_name}): {predicted_class}\nRecommended Pesticide: {recommended_pesticide}", font=("Times New Roman", 12, "bold"))

    # Display the input image below the buttons
    img = Image.open(file_path)
    img.thumbnail((400, 300)) 
    img_tk = ImageTk.PhotoImage(img)
    input_image_label.config(image=img_tk)
    input_image_label.image = img_tk

# Function to handle the "Browse Image" button
def browse_image():
    file_path = filedialog.askopenfilename(initialdir="/", title="Select Image",
                                           filetypes=(("Image files", "*.png;*.jpg;*.jpeg"), ("all files", "*.*")))
    if file_path:
        browse_label.config(text=f"Selected Image: {file_path}")
        # Enable plant type buttons
        sugarcane_button.config(state=tk.NORMAL)
        maize_button.config(state=tk.NORMAL)
        cotton_button.config(state=tk.NORMAL)
        rice_button.config(state=tk.NORMAL)
        wheat_button.config(state=tk.NORMAL)
        # Save file path for later use
        browse_image.file_path = file_path

# Create the main Tkinter window
root = tk.Tk()
root.title("Plant Disease Classification and Pesticide Recommendation")

root.geometry("700x750")

# Create widgets
browse_button = tk.Button(root, text="Browse Image", command=browse_image)
browse_label = tk.Label(root, text="Selected Image: ")

button_frame = tk.Frame(root)

sugarcane_button = tk.Button(button_frame, text="Sugarcane", command=lambda: classify_image('sugarcane', browse_image.file_path), state=tk.DISABLED)
maize_button = tk.Button(button_frame, text="Maize", command=lambda: classify_image('maize', browse_image.file_path), state=tk.DISABLED)
cotton_button = tk.Button(button_frame, text="Cotton", command=lambda: classify_image('cotton', browse_image.file_path), state=tk.DISABLED)
rice_button = tk.Button(button_frame, text="Rice", command=lambda: classify_image('rice', browse_image.file_path), state=tk.DISABLED)
wheat_button = tk.Button(button_frame, text="Wheat", command=lambda: classify_image('wheat', browse_image.file_path), state=tk.DISABLED)

result_label = tk.Label(root, text="Predicted Class: ", font=("Times New Roman", 11, "bold"))
input_image_label = tk.Label(root)

# Arrange widgets in the layout
browse_button.pack(pady=10)
browse_label.pack(pady=5)

button_frame.pack(pady=10)
sugarcane_button.pack(side="left", padx=5)
maize_button.pack(side="left", padx=5)
cotton_button.pack(side="left", padx=5)
rice_button.pack(side="left", padx=5)
wheat_button.pack(side="left", padx=5)

result_label.pack()
input_image_label.pack(pady=10)

# Run the Tkinter main loop
root.mainloop()
    

TypeError: Error when deserializing class 'KerasLayer' using config={'name': 'keras_layer_1', 'trainable': False, 'dtype': 'float32', 'batch_input_shape': [None, 224, 224, 3], 'handle': 'https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4'}.

Exception encountered: Trying to load a model of incompatible/unknown type. 'C:\Users\Admin\AppData\Local\Temp\tfhub_modules\145bb06ec3b59b08fb564ab752bd5aa222bfb50a' contains neither 'saved_model.pb' nor 'saved_model.pbtxt'.